In [242]:
# set the matplotlib backend so figures can be saved in the background
#TODO: cleanup, lint
import matplotlib
matplotlib.use("Agg")
 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
    
# import the necessary packages
from musicrec.vgg import VGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from datetime import datetime
import pickle
import cv2
import os
import sys
import tensorflow as tf

from keras.models import Sequential

import warnings
warnings.filterwarnings('ignore')

In [243]:
data_folder = Path("../../../audio/testfiles/GTZAN/genres/")
output_folder = Path("./output/cvnn.model")
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 2.97
# Matplotlib colormap for spectogram
spectogram_cmap = 'binary' 
# Predefined list of genres
pred_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() 
epochs = 12

randomseed = 11
#randomseed = datetime.now()
# Seed for RNG
random.seed(datetime.now())

In [244]:
#Get directories of all songs
songs = []
genres = []

spectograms = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [245]:
mel_specs = []  
labels = []
spectograms = []
for song in songs:
    y, sr = librosa.load(song, mono=True, duration=duration)
    m_sp = librosa.feature.melspectrogram(y=y, sr=sr)
    #if m_sp.shape != (128, 128): continue
    mel_specs.append(m_sp)
    label = song.parts[-2]
    labels.append(label)
    spectograms.append( (m_sp, label) )
#spectograms = list(zip(mel_specs, labels))
print("Total number of songs: ", len(mel_specs))
input_shape = m_sp.shape + (1,)
len(mel_specs)*2/3

Total number of songs:  1000


666.6666666666666

In [246]:
batch_size = input_shape[1] # use second dim as batch size, as it varies with duration

In [247]:
imagesize_x = np.shape(mel_specs)[1]
imagesize_y = np.shape(mel_specs)[2]
print("Spectogram dimensions: ", imagesize_x, "x", imagesize_y)

Spectogram dimensions:  128 x 128


In [248]:
# Shuffle the spectograms
random.shuffle(spectograms)
#spectograms[0][1]

In [249]:
#TODO: change
testsplit = len(mel_specs)*2/3
train = spectograms[:int(testsplit)]
test = spectograms[int(testsplit):]

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape(input_shape) for x in X_train])
X_test = np.array([x.reshape(input_shape) for x in X_test])

# One-Hot encoding for classes
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.transform(y_test)


"""
label_encoder = LabelEncoder()
y_train_int = label_encoder.fit_transform(y_train)
y_test_int = label_encoder.fit_transform(y_test)
#print(integer_encoded)

y_train_bin = np.array(keras.utils.to_categorical(y_train_int, 10))
y_test_bin = np.array(keras.utils.to_categorical(y_test_int, 10))
#print("Encoding of test data:\n", y_test, "=>\n", y_test_int, "=>\n", y_test_bin)
"""
0

0

In [250]:
y_train = y_train_bin
y_test = y_test_bin

In [251]:
"""

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2,horizontal_flip=True, 
                         fill_mode="nearest")
 
# initialize our VGG-like Convolutional Neural Network
model = VGGNet.build(width=imagesize_x, height=imagesize_y, depth=1, 
                          classes=len(lb.classes_))
"""
0

0

In [252]:
model = Sequential()

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

In [253]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

H = model.fit(
	x=X_train, 
	y=y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data= (X_test, y_test_bin))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 666 samples, validate on 334 samples
Epoch 1/12
666/666 [==============================] - 15s 22ms/step - loss: 8.3209 - acc: 0.0961 - val_loss: 3.0116 - val_acc: 0.1108
Epoch 2/12
666/666 [==============================] - 13s 20ms/step - loss: 4.0351 - acc: 0.1081 - val_loss: 2.3350 - val_acc: 0.1707
Epoch 3/12
666/666 [==============================] - 18s 26ms/step - loss: 2.8134 - acc: 0.1411 - val_loss: 2.3149 - val_acc: 0.1557
Epoch 4/12
666/666 [==============================] - 14s 21ms/step - loss: 2.4999 - acc: 0.1261 - val_loss: 2.2930 - val_acc: 0.1527
Epoch 5/12
666/666 [==============================] - 14s 22ms/step - loss: 2.3553 - acc: 0.1622 - val_loss: 2.2861 - val_acc: 0.1617
Epoch 6/12
666/666 [==============================] - 14s 22ms/step - loss: 2.3081 - acc: 0.1937 - val_loss: 2.2903 - val_acc: 0.1557
Epoch 7/12
666/666 [==============================] - 13s 20ms/step - loss: 2.2824 - acc: 0.1967 - val_loss: 2.2816 - val_acc: 0.1557
Epoch 8/12
666/6

In [254]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(X_test, batch_size=batch_size)
print(classification_report(y_test_bin.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
"""
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
"""
0

[INFO] evaluating network...
              precision    recall  f1-score   support

       blues       0.25      0.21      0.23        33
   classical       0.40      0.61      0.48        28
     country       0.14      0.28      0.19        29
       disco       0.18      0.06      0.09        33
      hiphop       0.21      0.32      0.26        37
        jazz       0.43      0.08      0.13        39
       metal       0.27      0.08      0.12        37
         pop       0.23      0.29      0.25        31
      reggae       0.06      0.14      0.08        28
        rock       0.00      0.00      0.00        39

   micro avg       0.19      0.19      0.19       334
   macro avg       0.22      0.21      0.18       334
weighted avg       0.22      0.19      0.18       334



0